In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00


## LOAD DATA

In [2]:
from datasets import load_dataset

dataset = load_dataset("sem_eval_2010_task_8")

print(dataset)
print(dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 2717
    })
})
{'sentence': 'The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'relation': 3}


## PRE-PROCESSING DATA

### QUESTIONS TEMPLATE

In [3]:
RELATION_LABELS = dataset["train"].features['relation'].names

questions = {
    "Cause-Effect(e1,e2)": ["What is caused by <e1>?", "What causes <e2>?"],
    "Cause-Effect(e2,e1)": ["What causes <e1>?", "What is caused by <e2>?"],
    "Instrument-Agency(e1,e2)": ["What uses <e1>?", "What is the instrument of <e2>?"],
    "Instrument-Agency(e2,e1)": ["What is the instrument of <e1>?", "What uses <e2>?"],
    "Product-Producer(e1,e2)": ["What produces <e1>?", "What is the product of <e2>?"],
    "Product-Producer(e2,e1)": ["What is the product of <e1>?", "What produces <e2>?"],
    "Content-Container(e1,e2)": ["What stores <e1>?", "What is stored inside <e2>?"],
    "Content-Container(e2,e1)": ["What is stored inside <e1>?", "What stores <e2>?"],
    "Entity-Origin(e1,e2)": ["What is the origin of <e1>?", "What entity originates from <e2>?"],
    "Entity-Origin(e2,e1)": ["What entity originates from <e1>?", "What is the origin of <e2>?"],
    "Entity-Destination(e1,e2)": ["What is the destination of <e1>?", "What entity is moving toward <e2>?"],
    "Entity-Destination(e2,e1)": ["What entity is moving toward <e1>?", "What is the destination of <e2>?"],
    "Component-Whole(e1,e2)": ["What entity does <e1> operate within?", "What functional component of <e2> is mentioned?"],
    "Component-Whole(e2,e1)": ["What functional component of <e1> is mentioned?", "What entity does <e2> operate within?"],
    "Member-Collection(e1,e2)": ["Which collection/organization does <e1> belong to?", "Who are the members of <e2>?"],
    "Member-Collection(e2,e1)": ["Who are the members of <e1>?", "Which collection/organization does <e2> belong to?"],
    "Message-Topic(e1,e2)": ["What is the main topic discussed in <e1>?", "Which message contains information about <e2>?"],
    "Message-Topic(e2,e1)": ["Which message contains information about <e1>?", "What is the main topic discussed in <e2>?"],
}

In [4]:
print(RELATION_LABELS[14])
print(RELATION_LABELS.index('Instrument-Agency(e2,e1)'))
print(RELATION_LABELS)

Message-Topic(e1,e2)
11
['Cause-Effect(e1,e2)', 'Cause-Effect(e2,e1)', 'Component-Whole(e1,e2)', 'Component-Whole(e2,e1)', 'Content-Container(e1,e2)', 'Content-Container(e2,e1)', 'Entity-Destination(e1,e2)', 'Entity-Destination(e2,e1)', 'Entity-Origin(e1,e2)', 'Entity-Origin(e2,e1)', 'Instrument-Agency(e1,e2)', 'Instrument-Agency(e2,e1)', 'Member-Collection(e1,e2)', 'Member-Collection(e2,e1)', 'Message-Topic(e1,e2)', 'Message-Topic(e2,e1)', 'Product-Producer(e1,e2)', 'Product-Producer(e2,e1)', 'Other']


### RELATION LIMIT STRATEGIES

In [5]:
## uncomment this for filtering top n relation

# import gensim.downloader as api
# import numpy as np

# word2vec = api.load("word2vec-google-news-300")

# def get_embedding(word):
#     total = np.random.uniform(-0.25, 0.25, word2vec.vector_size)
#     words = word.split(' ')

#     for word in words:
#         if word in word2vec:
#             total += word2vec[word]

#     if np.all(total == 0):
#         return total

#     return total/len(words)

In [6]:
## uncomment this for filtering top n relation

# relation_embeddings = {}

# for rel in questions.keys():
#     base_rel = rel.replace('(e1,e2)', '').replace('(e2,e1)', '')

#     if base_rel not in relation_embeddings:
#         rel1, rel2 = base_rel.split("-")

#         rel1_emb = get_embedding(rel1)
#         rel2_emb = get_embedding(rel2)

#         # Store both relation directions
#         relation_embeddings[f"{rel1}-{rel2}(e1,e2)"] = rel1_emb - rel2_emb  # e1 - e2
#         relation_embeddings[f"{rel1}-{rel2}(e2,e1)"] = rel2_emb - rel1_emb  # e2 - e1

In [7]:
import random

def select_question(relation_label):
    if relation_label == "Other":
        selected_relations = random.sample(list(questions.keys()), 2)
    else:
        selected_relations = [relation_label, random.choice([rel for rel in questions if rel != relation_label])]

    return {rel: questions[rel] for rel in selected_relations}

def select_all_question(relation_label):
    return questions

## uncomment this for filtering top n relation
# def select_question_filter_embedding(e1, e2, top_n, filter_type):
#     e1_emb = get_embedding(e1)
#     e2_emb = get_embedding(e2)
#     entity_diff = e1_emb - e2_emb

#     # print(e1_emb.shape)
#     # print(e2_emb.shape)
#     # print(entity_diff.shape)

#     scores = []
#     for rel, rel_diff in relation_embeddings.items():
#         if filter_type == 'cosine':
#             similarity = np.dot(entity_diff, rel_diff) / (np.linalg.norm(entity_diff) * np.linalg.norm(rel_diff))
#         elif filter_type == 'euclidean':
#             similarity = -np.linalg.norm(entity_diff - rel_diff)
#         else:
#             similarity = None

#         scores.append((rel, similarity))
#     scores.sort(key=lambda x: x[1], reverse=True)

#     # print(scores)

#     selected_relations = {}
#     for rel, _ in scores[:top_n]:
#         if rel in questions:
#             # print('----', rel, '-----')
#             selected_relations[rel] = questions[rel]

#     return selected_relations  # Returns a list of (relation, questions)

### PREPAREING DATA

In [8]:
import re
from datasets import Dataset
import time
from tqdm import tqdm

def prepare_data(dataset_type, choose_question_func):
    data = {
        'sentence': [],
        'relation': [],
        'relation_label': [],
        'question': [],
        'answer': [],
        'answer_position': [],
        'other_entity': [],
        'question_relation_id': [],
    }

    total_rows = len(dataset[dataset_type])
    start_time = time.time()

    for i, row in enumerate(tqdm(dataset[dataset_type], desc="Processing rows", unit="row")):
        e1_start = row['sentence'].index("<e1>") + 1
        e1_end = row['sentence'].index("</e1>") - 3
        e2_start = row['sentence'].index("<e2>") - 6
        e2_end = row['sentence'].index("</e2>") - 10
        entities_position = {
            'e1': [e1_start, e1_end],
            'e2': [e2_start, e2_end]
        }

        sentence = re.sub(r"</?e[12]>", " ", row['sentence'])
        relation = row['relation']
        relation_label = RELATION_LABELS[relation]

        # Extract e1 and e2
        e1_match = re.search(r"<e1>(.*?)</e1>", row['sentence'])
        e2_match = re.search(r"<e2>(.*?)</e2>", row['sentence'])
        entities = {
            'e1': e1_match.group(1) if e1_match else None,
            'e2': e2_match.group(1) if e2_match else None
        }

        choosen_questions = choose_question_func(relation_label)
        # choosen_questions = choose_question_func(entities['e1'], entities['e2'], 10, 'euclidean')  ## uncomment this for filtering top n relation

        # print(choosen_questions) ## Checking

        for rel, q_list in choosen_questions.items():
            for q in q_list:
                question = q.replace('<e1>', entities['e1']).replace('<e2>', entities['e2'])

                is_e1_in_question = entities['e1'] in question

                if relation_label == rel:
                    if relation != 18:
                        answer = entities['e2'] if is_e1_in_question else entities['e1']
                        answer_position = entities_position['e2'] if is_e1_in_question else entities_position['e1']
                    else:
                        answer = 'other'
                        answer_position = [0, 0]
                else:
                    answer = None
                    answer_position = None

                other_entity = entities['e1'] if is_e1_in_question else entities['e2']

                data['sentence'].append(sentence)
                data['relation'].append(relation)
                data['relation_label'].append(relation_label)
                data['question'].append([question, rel])
                data['answer'].append(answer)
                data['answer_position'].append(answer_position)
                data['other_entity'].append(other_entity)
                data['question_relation_id'].append(RELATION_LABELS.index(rel))

    elapsed_time = time.time() - start_time
    print(f"Completed {total_rows} rows in {elapsed_time:.2f}s")

    return Dataset.from_dict(data)

In [9]:
## Choose 4 questions (2 related + 2 random unrelateed)
prepared_train_dataset = prepare_data('train', select_question)
prepared_test_dataset = prepare_data('test', select_all_question)

## Uncomment for no limit
# prepared_train_dataset = prepare_data('train', select_all_question)
# prepared_test_dataset = prepare_data('test', select_all_question)

## uncomment this for filtering top n relation
# prepared_train_dataset = prepare_data('train', select_question_filter_embedding)
# prepared_test_dataset = prepare_data('test', select_question_filter_embedding)

Processing rows: 100%|██████████| 8000/8000 [00:00<00:00, 20409.53row/s]


Completed 8000 rows in 0.39s


Processing rows: 100%|██████████| 2717/2717 [00:00<00:00, 8910.69row/s]


Completed 2717 rows in 0.31s


In [10]:
from datasets import Dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_6L_768D")

def tokenised_dataset(row):

    tokens = tokenizer(
        row['sentence'],
        row['question'][0],
        return_tensors="pt",
        truncation=True,
        max_length=80,
        padding="max_length",
        return_offsets_mapping=True
    )

    # Map to token positions
    offsets = tokens["offset_mapping"][0].tolist()
    start_token, end_token = 0, 0

    if row['question'][1] == row['relation_label']:
        for idx, (start, end) in enumerate(offsets):
            if start == row['answer_position'][0]:
                start_token = idx
            if end == row['answer_position'][1]:
                end_token = idx
                break

    return {
        'input_ids': tokens["input_ids"].squeeze(0),
        'attention_mask': tokens["attention_mask"].squeeze(0),
        'token_type_ids': tokens["token_type_ids"].squeeze(0),
        'start': start_token,
        'end': end_token,
        'relation': row['relation']
    }


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [11]:
tokenized_qa_train = prepared_train_dataset.map(tokenised_dataset, batched=False)
tokenized_qa_test = prepared_test_dataset.map(tokenised_dataset, batched=False)

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/97812 [00:00<?, ? examples/s]

In [12]:
print(tokenized_qa_train[0])
print(tokenized_qa_test[0])

{'sentence': 'The system as described above has its greatest application in an arrayed  configuration  of antenna  elements .', 'relation': 3, 'relation_label': 'Component-Whole(e2,e1)', 'question': ['What functional component of configuration is mentioned?', 'Component-Whole(e2,e1)'], 'answer': 'elements', 'answer_position': [101, 109], 'other_entity': 'configuration', 'question_relation_id': 3, 'input_ids': [101, 1996, 2291, 2004, 2649, 2682, 2038, 2049, 4602, 4646, 1999, 2019, 9140, 2098, 9563, 1997, 13438, 3787, 1012, 102, 2054, 8360, 6922, 1997, 9563, 2003, 3855, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 

In [13]:
# Removing unnecessary column
train_dataset = tokenized_qa_train.remove_columns(
    ['sentence', 'relation_label', 'question', 'answer', 'answer_position', 'other_entity']
)

test_dataset = tokenized_qa_test.remove_columns(
    ['sentence', 'relation_label', 'question', 'answer', 'answer_position', 'other_entity']
)

print(train_dataset[0])
print(test_dataset[0])

{'relation': 3, 'question_relation_id': 3, 'input_ids': [101, 1996, 2291, 2004, 2649, 2682, 2038, 2049, 4602, 4646, 1999, 2019, 9140, 2098, 9563, 1997, 13438, 3787, 1012, 102, 2054, 8360, 6922, 1997, 9563, 2003, 3855, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'start': 17, 'end': 17}
{'relation': 14, 'question_relation_id': 0, 'input_ids': [101, 1996, 2087

In [14]:
import torch
from transformers import AutoModel

# Load TinyBERT model
model_name = "huawei-noah/TinyBERT_General_6L_768D"
tinybert = AutoModel.from_pretrained(model_name)

class TinyBERTForQA(torch.nn.Module):
    def __init__(self, tinybert):
        super().__init__()
        self.tinybert = tinybert
        self.qa_outputs = torch.nn.Linear(768, 2)  # Add QA head for start and end logits

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.tinybert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = self.qa_outputs(outputs.last_hidden_state)
        start_logits, end_logits = logits.split(1, dim=-1)  # Split into start and end logits
        return start_logits.squeeze(-1), end_logits.squeeze(-1)

# Initialise new model
model = TinyBERTForQA(tinybert)

pytorch_model.bin:   0%|          | 0.00/287M [00:00<?, ?B/s]

### PREPARE THE INPUT

In [15]:
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(item["attention_mask"], dtype=torch.long),
            "token_type_ids": torch.tensor(item["token_type_ids"], dtype=torch.long),
            "start_positions": torch.tensor(item["start"], dtype=torch.long),
            "end_positions": torch.tensor(item["end"], dtype=torch.long),
            "relation": torch.tensor(item["relation"] , dtype=torch.long),
            "question_relation_id": torch.tensor(item["question_relation_id"], dtype=torch.long)
        }

In [16]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

train_dataset_prepare = QADataset(train_dataset)
train_dataloader = DataLoader(train_dataset_prepare, batch_size=128, shuffle=True, collate_fn=data_collator)

test_dataset_prepare = QADataset(test_dataset)
test_dataloader = DataLoader(test_dataset_prepare, batch_size=128, shuffle=False, collate_fn=data_collator)

## TRAINING

In [17]:
import torch
import torch.nn as nn
from transformers import AdamW

# Ensure model and optimizer are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

epochs = 10
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Move batch to GPU
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch["token_type_ids"]
        )

        start_logits, end_logits = outputs

        # Compute loss
        loss_start = loss_fn(start_logits, batch["start_positions"])
        loss_end = loss_fn(end_logits, batch["end_positions"])

        loss = loss_start + loss_end

        # Backpropagation
        loss.backward()
        optimizer.step()

        a = loss.item()
        # print(a)
        total_loss += a

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Loss: {avg_loss}")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


model.safetensors:   0%|          | 0.00/287M [00:00<?, ?B/s]

Epoch 1 - Loss: 2.637818471431732
Epoch 2 - Loss: 1.1638281667232513
Epoch 3 - Loss: 0.8132317345142365
Epoch 4 - Loss: 0.6296043474674224
Epoch 5 - Loss: 0.5078891949057579
Epoch 6 - Loss: 0.4276498036980629
Epoch 7 - Loss: 0.3524037138223648
Epoch 8 - Loss: 0.3034774996638298
Epoch 9 - Loss: 0.2525460143089294
Epoch 10 - Loss: 0.21554833509027957


## SAVE MODEL

In [18]:
torch.save(model.state_dict(), "relation_extraction_model.pth")
model.load_state_dict(torch.load("relation_extraction_model.pth"))

<ipython-input-18-cf2489ba4aa0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("relation_extraction_model.pth"))


<All keys matched successfully>

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')
# save_path = "/content/drive/My Drive/relation_extraction_model_final.pth"
# torch.save(model.state_dict(), save_path)
# model.load_state_dict(torch.load("/content/drive/My Drive/relation_extraction_model_final.pth"))

## EVALUATION

In [20]:
import torch

test_data = []
model.eval()

with torch.no_grad():
    for test_batch in test_dataloader:
        test_batch = {k: v.to(device) for k, v in test_batch.items()}

        test_outputs = model(
            input_ids=test_batch["input_ids"],
            attention_mask=test_batch["attention_mask"],
            token_type_ids=test_batch["token_type_ids"]
        )

        test_start_logits, test_end_logits = test_outputs

        # Get predictions
        test_pred_start = torch.argmax(test_start_logits, dim=1)
        test_pred_end = torch.argmax(test_end_logits, dim=1)

        # Store data in structured format
        for i in range(test_batch["start_positions"].size(0)):
            test_data.append({
                "pos_rel": (test_batch["start_positions"][i].item(), test_batch["end_positions"][i].item()),
                "pos_pred": (test_pred_start[i].item(), test_pred_end[i].item()),
                "relation": test_batch["relation"][i].item(),
                "question_relation": test_batch["question_relation_id"][i].item()
            })

In [21]:
#TEST
# for each in test_data[36*1:36*5]:
#   if each['pos_rel'] == each['pos_pred'] and each['pos_rel'] != (0, 0):
#     print(each)

In [22]:
print(len(test_data))

97812


In [23]:
def calculate_relation_accuracy(test_data):
    pred_relations = []
    real_relations = []
    for i in range(0, len(test_data), 36):
        have_relation = False
        for each in test_data[i:i+36]:
            pos_rel = each['pos_rel']
            pos_pred = each['pos_pred']
            if pos_rel == pos_pred and pos_rel != (0,0):
                real_relations.append(each['relation'])
                pred_relations.append(each['question_relation'])
                have_relation = True
                break
        if not have_relation:  # If no relation was found
            pred_relations.append(0)
            real_relations.append(each['relation'])

    return pred_relations, real_relations

pred_relations, real_relations = calculate_relation_accuracy(test_data)

In [24]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Calculate evaluation metrics
macro_f1 = f1_score(real_relations, pred_relations, average="macro")
precision = precision_score(real_relations, pred_relations, average="macro", zero_division=0)
recall = recall_score(real_relations, pred_relations, average="macro", zero_division=0)
accuracy = accuracy_score(real_relations, pred_relations)

# Print results
print(f"Macro F1 Score: {macro_f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")

Macro F1 Score: 0.8232693012334161
Precision: 0.8519415694046832
Recall: 0.8279945237515587
Accuracy: 0.7854251012145749
